This is a live notebook with experimental code to develop analysis investigating the correlation between features.

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import itertools
import json

import scipy.stats as spstats
# fourier transform
from scipy.fft import fft, ifft

from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
# from statsmodels.tsa.api import acf, graphics, pacf
from statsmodels.tsa.ar_model import AutoReg
# from statsmodels.tsa.ar_model import ar_select_order

import os
print(os.listdir("."))


import re
res_digit = r'[0-9]'


['debug_windowing_data.ipynb', 'testing_mlpwrapper.ipynb', 'logs', 'gait_data_features.ipynb', 'feature_development.ipynb', 'data', 'feature_analysis.ipynb', 'merging_dataframes.ipynb', 'data_and_processing_description.ipynb', 'featured_prediction_random_forest.ipynb', 'window_timings_and_IBI.ipynb', 'gait_data_exploration.ipynb', 'Analyse_results.ipynb', '.ipynb_checkpoints', 'loading_e4_data.ipynb', 'data_investigation_scratch.ipynb', 'simply_load_and_inspect_data.ipynb', 'grouping_and_crossvalidation.ipynb', 'sample-entropy-numba-impl.ipynb', 'featured_prediction.ipynb']


In [2]:
# This is a hack to make the library in the parent folder available for imoprts
# A better solution is by np8 here:
# https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder
import sys
import os
import inspect

thisdir = sys.path[0]
print(f"thisdir = {thisdir}")
parentdir = os.path.dirname(thisdir)
#print(f"parentdir = {parentdir}")
if not parentdir in sys.path:
    print("Adding parent directory to python path")
    sys.path.insert(1, parentdir)
else:
    print("Skipping adding parent direct to path (there already)")

print(f"sys.path =\n{sys.path}")



thisdir = /home/luke/git/external/predicament/notebooks
Adding parent directory to python path
sys.path =
['/home/luke/git/external/predicament/notebooks', '/home/luke/git/external/predicament', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/luke/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/lib/python3.10/dist-packages']


In [3]:
## ensure relative path to data directory is sound
# for the notebook we need to modify the BASE_DATA_FOLDER
import os 
os.environ['PREDICAMENT_DATA_DIR'] =  '../data'

In [4]:
from predicament.utils.config import DREEM_EEG_CHANNELS
from predicament.utils.config import FEATURED_BASE_PATH
from predicament.utils.config import WINDOWED_BASE_PATH
from predicament.utils.file_utils import drop_nan_cols
from predicament.utils.file_utils import load_windowed_data_and_config
from predicament.utils.file_utils import load_dataframe_and_config
from predicament.utils.file_utils import write_dataframe_and_config

from predicament.data.timeseries import create_participant_data_edf_only
from predicament.data.windowed import window_all_participants_data
from predicament.data.windowed import merge_condition_data
from predicament.data.partitioning import between_subject_cv_partition

from predicament.data.features import MAXIMAL_FEATURE_GROUP
from predicament.data.features import STATS_FEATURE_GROUP
from predicament.data.features import INFO_FEATURE_GROUP
from predicament.data.features import FREQ_FEATURE_GROUP
from predicament.data.features import convert_timeseries_to_features
from predicament.data.features import filter_features


In [5]:
from predicament.utils.config import E4_CSV_FILES
from predicament.utils.config import E4_FULL_DIRPATHS
E4_FULL_DIRPATHS

{'VG_01': '../data/CARE_HOME_DATA/./VG01/E4_8921_15_44/',
 'VG_03': '../data/CARE_HOME_DATA/./VG03/E4_9921_12_16/',
 'VG_05': '../data/CARE_HOME_DATA/./VG05/E4_9921_13_24/',
 'VG_06': '../data/CARE_HOME_DATA/./VG06/E4_51021_13_33/',
 'VG_07': '../data/CARE_HOME_DATA/./VG07/E4_51021_15_39/',
 'VG_08': '../data/CARE_HOME_DATA/./VG08/E4_71021_10_42/',
 'VG_09': '../data/CARE_HOME_DATA/./VG09/E4_11221_14_46/',
 'VG_10': '../data/CARE_HOME_DATA/./VG10/E4_31221_11_17/',
 'VH_01': '../data/CARE_HOME_DATA/./VH01/E4_61021_11_03/',
 'VH_02': '../data/CARE_HOME_DATA/./VH02/E4_61021_13_59/',
 'VH_03': '../data/CARE_HOME_DATA/./VH03/E4_11221_11_22/'}

In [6]:
subdir = 'ethan'
df, config = load_windowed_data_and_config(subdir)
df

Reading dataframe from ../data/windowed/ethan/windowed.csv
Reading config from ../data/windowed/ethan/details.cfg


,participant,condition,window index,EEG Fpz-O1[0],EEG Fpz-O1[1],EEG Fpz-O1[2],EEG Fpz-O1[3],EEG Fpz-O1[4],EEG Fpz-O1[5],EEG Fpz-O1[6],...,EEG Fpz-O2[22],EEG Fpz-O2[23],EEG Fpz-O2[24],EEG Fpz-O2[25],EEG Fpz-O2[26],EEG Fpz-O2[27],EEG Fpz-O2[28],EEG Fpz-O2[29],EEG Fpz-O2[30],EEG Fpz-O2[31]
0,VG_01,0,0,0.000012,0.000015,0.000011,0.000003,-0.000006,-0.000015,-0.000019,...,0.000073,0.000069,0.000067,0.000067,0.000066,5.707637e-05,0.000045,0.000036,0.000036,0.000042
1,VG_01,0,1,0.000047,0.000050,0.000038,0.000023,0.000019,0.000030,0.000048,...,0.000012,0.000016,0.000012,-0.000002,-0.000021,-3.745327e-05,-0.000045,-0.000042,-0.000033,-0.000023
2,VG_01,0,2,-0.000042,-0.000039,-0.000029,-0.000016,-0.000009,-0.000009,-0.000015,...,0.000062,0.000061,0.000050,0.000032,0.000011,-7.820249e-06,-0.000018,-0.000019,-0.000018,-0.000022
3,VG_01,0,3,-0.000048,-0.000051,-0.000051,-0.000051,-0.000049,-0.000045,-0.000038,...,-0.000040,-0.000033,-0.000025,-0.000017,-0.000012,-9.346151e-06,-0.000008,-0.000009,-0.000010,-0.000010
4,VG_01,0,4,-0.000007,-0.000007,-0.000014,-0.000027,-0.000043,-0.000055,-0.000057,...,-0.000011,-0.000012,-0.000010,-0.000003,0.000003,3.349355e-06,-0.000001,-0.000007,-0.000007,-0.000003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10776,VG_01,2,1870,0.000008,0.000010,0.000006,-0.000003,-0.000012,-0.000016,-0.000012,...,0.000013,0.000016,0.000016,0.000011,0.000004,-3.044175e-06,-0.000010,-0.000016,-0.000018,-0.000013
10777,VG_01,2,1871,0.000028,0.000024,0.000018,0.000011,0.000007,0.000004,0.000004,...,-0.000023,-0.000018,-0.000010,-0.000001,0.000007,1.116197e-05,0.000012,0.000011,0.000007,0.000002
10778,VG_01,2,1872,0.000029,0.000026,0.000021,0.000015,0.000006,-0.000004,-0.000010,...,0.000018,0.000016,0.000014,0.000010,0.000003,-3.242542e-06,-0.000005,-0.000002,0.000003,0.000002
10779,VG_01,2,1873,-0.000024,-0.000023,-0.000026,-0.000032,-0.000039,-0.000046,-0.000052,...,0.000003,-0.000002,-0.000007,-0.000009,-0.000006,4.043641e-07,0.000010,0.000019,0.000027,0.000033


In [9]:
list(df[df['condition']==0]['window index'].to_numpy())


[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [11]:
channel_cols = [c for c in df.columns if c.startswith('EEG Fpz-O1')]
df[channel_cols].to_

,EEG Fpz-O1[0],EEG Fpz-O1[1],EEG Fpz-O1[2],EEG Fpz-O1[3],EEG Fpz-O1[4],EEG Fpz-O1[5],EEG Fpz-O1[6],EEG Fpz-O1[7],EEG Fpz-O1[8],EEG Fpz-O1[9],...,EEG Fpz-O1[22],EEG Fpz-O1[23],EEG Fpz-O1[24],EEG Fpz-O1[25],EEG Fpz-O1[26],EEG Fpz-O1[27],EEG Fpz-O1[28],EEG Fpz-O1[29],EEG Fpz-O1[30],EEG Fpz-O1[31]
0,0.000012,0.000015,0.000011,0.000003,-0.000006,-0.000015,-0.000019,-0.000015,-0.000006,0.000003,...,0.000017,0.000009,0.000007,0.000012,1.691463e-05,0.000015,9.010452e-06,0.000006,0.000014,0.000032
1,0.000047,0.000050,0.000038,0.000023,0.000019,0.000030,0.000048,0.000058,0.000051,0.000032,...,-0.000031,-0.000026,-0.000022,-0.000022,-2.507820e-05,-0.000029,-3.049516e-05,-0.000030,-0.000032,-0.000037
2,-0.000042,-0.000039,-0.000029,-0.000016,-0.000009,-0.000009,-0.000015,-0.000023,-0.000032,-0.000038,...,0.000008,0.000016,0.000015,0.000009,-2.822919e-07,-0.000008,-1.458000e-05,-0.000022,-0.000031,-0.000041
3,-0.000048,-0.000051,-0.000051,-0.000051,-0.000049,-0.000045,-0.000038,-0.000031,-0.000029,-0.000036,...,-0.000041,-0.000053,-0.000058,-0.000055,-4.718853e-05,-0.000040,-3.434043e-05,-0.000030,-0.000023,-0.000014
4,-0.000007,-0.000007,-0.000014,-0.000027,-0.000043,-0.000055,-0.000057,-0.000047,-0.000031,-0.000018,...,-0.000022,-0.000021,-0.000025,-0.000034,-4.404517e-05,-0.000050,-4.686809e-05,-0.000037,-0.000027,-0.000022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10776,0.000008,0.000010,0.000006,-0.000003,-0.000012,-0.000016,-0.000012,-0.000004,0.000006,0.000015,...,0.000042,0.000034,0.000024,0.000013,7.225147e-06,0.000008,1.351186e-05,0.000021,0.000027,0.000029
10777,0.000028,0.000024,0.000018,0.000011,0.000007,0.000004,0.000004,0.000008,0.000013,0.000019,...,0.000009,0.000001,-0.000003,-0.000002,4.127565e-06,0.000011,1.851682e-05,0.000024,0.000028,0.000030
10778,0.000029,0.000026,0.000021,0.000015,0.000006,-0.000004,-0.000010,-0.000007,0.000002,0.000011,...,-0.000021,-0.000017,-0.000012,-0.000011,-1.412222e-05,-0.000022,-2.953384e-05,-0.000033,-0.000032,-0.000028
10779,-0.000024,-0.000023,-0.000026,-0.000032,-0.000039,-0.000046,-0.000052,-0.000055,-0.000056,-0.000055,...,0.000007,0.000010,0.000009,0.000007,7.179370e-06,0.000008,7.530327e-06,0.000002,-0.000005,-0.000010
